### 获取订单号的参数

```python

qm = QHJMall(MALL_KEY)
qm.get_order(order_type,**kargs)


# 请求类型 0 表示所有订单 1 未发货  2 已发货 3 已完成 4 申请退款 5 退款成功  10 未支付
"order_status": kargs.get("order_status", 0),
"order_time": kargs.get("order_time", ""), # today yesterday week month
"order_id": kargs.get("order_id", ""), # 订单号
"order_type": order_type,  # original free
"page": kargs.get("page", 1),  #

```


In [1]:
import re
import sys
sys.path.append(r"D:\往期\QHJ\ZERO")
sys.path.append(r"E:\dataparse\Python_DATA_PARSE\QHJ\ZERO")
import warnings
warnings.filterwarnings("ignore")

from pyquery import PyQuery as pq
import requests
from urllib.parse import urlencode,parse_qs
import numpy as np
import pandas as pd
import redis

import json
from tools import *

qm = QHJMall(MALL_KEY)


import datetime



In [85]:
l = []
# for order in qm.gen_orders("free"):
#     l.append(order)
for order in qm.gen_orders("original"):
    l.append(order)

请求 {'order_status': 0, 'order_time': '', 'order_id': '', 'order_type': 'original', 'page': 1}


In [14]:
data = pd.DataFrame(l)

In [90]:
data.columns

Index(['#', 'order_type', '下单时间', '原价', '备注', '操作', '支付时间', '支付金额', '数量',
       '用户信息', '订单号', '订单状态', '订单详情', '购买时折扣'],
      dtype='object')

In [91]:
free_df = data[data['order_type'] == "original"]

In [92]:
# 拆分为原子
f_df_r = free_df.drop(columns='订单详情').join(free_df['订单详情'].str.split('\n',expand=True).stack().reset_index(level=1, drop=True).rename("详情"))

In [94]:
f_df_r.columns

Index(['#', 'order_type', '下单时间', '原价', '备注', '操作', '支付时间', '支付金额', '数量',
       '用户信息', '订单号', '订单状态', '购买时折扣', '详情'],
      dtype='object')

In [95]:
# 提取数据
df_t = pd.concat([f_df_r[['订单号']], f_df_r['详情'].str.extract(
"""
(?P<商品名>.+?)(?=（规格[：:])（
规格[：:](?P<规格>.+?)(?=[，,]商品ID[：:])
[，,]商品ID[：:](?P<商品ID>.+)）\sX(?P<数量>\s\d+)""",flags=re.X)],axis=1)
df_t['发货商ID'] = df_t['商品ID'].str[:5]
df_t['SPUID'] = df_t['商品ID'].str[:-6]
df_t.reset_index(drop=True,inplace=True)
# df_t['数量'] = df_t['数量'].astype(str)
df_t = df_t[["发货商ID",'SPUID','商品ID','规格','数量','订单号']]

In [97]:
# 可发货分类
def tt(row):
    r = row.copy()
    print(r['数量'])
    s = '；'.join(r['规格'] + " X " +r['数量'])

    r.iloc[0]['规格'] = s
    print(row)
    print("="*30)
    return r.iloc[0]
    
df_t.groupby(['发货商ID','SPUID']).apply(tt)

3     6
4     6
Name: 数量, dtype: object
   发货商ID            SPUID                   商品ID 规格  数量               订单号
3  S0001  S0001P0001C0002  S0001P0001C0002P01E01  黑   6  3385063409686242
4  S0001  S0001P0001C0002  S0001P0001C0002P02E01  红   6  3385063409686242
3     6
4     6
Name: 数量, dtype: object
   发货商ID            SPUID                   商品ID 规格  数量               订单号
3  S0001  S0001P0001C0002  S0001P0001C0002P01E01  黑   6  3385063409686242
4  S0001  S0001P0001C0002  S0001P0001C0002P02E01  红   6  3385063409686242
2     7
Name: 数量, dtype: object
   发货商ID            SPUID                   商品ID  规格  数量               订单号
2  S0001  S0001P0005C0003  S0001P0005C0003P01E01  单耳   7  3385063409686242
0     6
1     8
Name: 数量, dtype: object
   发货商ID            SPUID                   商品ID      规格  数量               订单号
0  S0002  S0002P0001C0002  S0002P0001C0002P04E01   银 64G   6  3385063409686242
1  S0002  S0002P0001C0002  S0002P0001C0002P07E01  绿 256G   8  3385063409686242


发货商ID            SPUID                   商品ID  \
发货商ID SPUID                                                            
S0001 S0001P0001C0002  S0001  S0001P0001C0002  S0001P0001C0002P01E01   
      S0001P0005C0003  S0001  S0001P0005C0003  S0001P0005C0003P01E01   
S0002 S0002P0001C0002  S0002  S0002P0001C0002  S0002P0001C0002P04E01   

                                           规格  数量               订单号  
发货商ID SPUID                                                          
S0001 S0001P0001C0002           黑 X  6；红 X  6   6  3385063409686242  
      S0001P0005C0003                 单耳 X  7   7  3385063409686242  
S0002 S0002P0001C0002  银 64G X  6；绿 256G X  8   6  3385063409686242

In [11]:
stop

NameError: name 'stop' is not defined

In [ ]:
l = []
for order in qm.gen_orders("free",order_time='today'):
    l.append(order)

In [ ]:
pd.DataFrame(l).to_excel(r"C:\Users\qhj01\Desktop\test.xlsx")

In [ ]:
# 正式
conn = redis.Redis(**REDIS_MALL_ORDER_DIC)

# 测试



In [ ]:
sku_goods = pd.read_excel(r'D:\奇货居\素材\商城图片素材\商品信息.xlsx',sheet_name='商品详情') \
                [['商品ID','发货商ID','goods_type','规格','单位', '市场价','售价','商品名简称']] \
                .rename(columns={"售价":"单价","商品名简称":"商品名"}).fillna({"goods_type":"original"})

free_goods = sku_goods[sku_goods['goods_type']=='free']

order_kargs = {
    "order_status":1,
    "order_time":"yesterday"
}

yd_data = qm.ext_order(order_kargs,sku_goods,free_goods)

order_kargs = {
    "order_status":1,
    "order_time":"today"
}
to_data = qm.ext_order(order_kargs,sku_goods,free_goods)

In [ ]:

def uprow(row,):
    # 判断是否在已经 发过此物
    if  conn.sismember('order:old' ,row['订单号']):
        row['订单号'] = None
        
    return row



save_dir = r"D:\Downloads\QHJ_MALL"
save = SaveXl(save_dir)

data = pd.concat([to_data.apply(uprow,axis=1), #
                  yd_data.apply(uprow,axis=1)])
data.dropna(subset=['订单号'],inplace=True)
data.drop_duplicates(subset='订单号',inplace=True)

In [ ]:
len(data)

In [ ]:



save(data,"订单")
%run D:\往期\QHJ\ZERO\app\process_order.py


In [ ]:


stop


In [ ]:
spp_path =  os.path.join(r"D:\奇货居\素材\商城图片素材","发货商详情.xlsx")
spp_data = pd.read_excel(spp_path)

In [ ]:
spp_data

In [ ]:
stop

In [ ]:
d = dict(zip(spp_data['发货商ID'],spp_data['发货商'],))

d

In [ ]:
conn.keys('ORDER:NEW:TEMP*')

In [ ]:
conn = redis.Redis(**REDIS_MALL_ORDER_DIC)

conn.keys('ORDER:NEW:TEMP*')
src_new = 'ORDER:NEW:TEMP:S0003'
dst_new = src_new.replace(":TEMP","")
conn.sunionstore( dst_new,dst_new,src_new)
conn.delete(src_new)

In [ ]:
# conn.sunionstore('ORDER:NEW',"ORDER:NEW_TEMP",'ORDER:NEW')

# conn.sadd("ORDER:NEW","zero")
# conn.sadd("ORDER:WB","zero")
# conn.sadd("ORDER:NEW_TEMP","zero")